In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from extract_feats import *

In [3]:
df = pd.read_csv('data/training.csv')
df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


In [4]:
df_test = pd.read_csv('data/test.csv')
df_test

,text,label
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0
...,...,...
1995,i just keep feeling like someone is being unki...,3
1996,im feeling a little cranky negative after this...,3
1997,i feel that i am useful to my people and that ...,1
1998,im feeling more comfortable with derby i feel ...,1


In [39]:
label_mapping = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [5]:
'''
features = {'anger': 0, 'anticipation': 1, 'disgust': 2, 'fear': 3, 'joy': 4, 'sadness': 5,
            'surprise': 6, 'trust': 7, 'negative': 8, 'positive': 9, 'CC': 10, 'IN': 11, 'JJR': 12,
            'JJS': 13, 'PRP': 14, '!': 15, '?': 16}
nrc_pos_feats = extract_nrc_pos_feats(df, features)
nrc_pos_feats
'''

"\nfeatures = {'anger': 0, 'anticipation': 1, 'disgust': 2, 'fear': 3, 'joy': 4, 'sadness': 5,\n            'surprise': 6, 'trust': 7, 'negative': 8, 'positive': 9, 'CC': 10, 'IN': 11, 'JJR': 12,\n            'JJS': 13, 'PRP': 14, '!': 15, '?': 16}\nnrc_pos_feats = extract_nrc_pos_feats(df, features)\nnrc_pos_feats\n"

In [6]:
cleaned_data = cleanData(df)
cleaned_data

,text,label
0,didnt feel humili,0
1,go feel hopeless damn hope around someon care ...,0
2,im grab minut post feel greedi wrong,3
3,ever feel nostalg fireplac know still properti,2
4,feel grouchi,3
...,...,...
15995,veri brief time beanbag said anna feel like be...,0
15996,turn feel pathet still wait tabl sub teach degre,0
15997,feel strong good overal,1
15998,feel like thi wa rude comment im glad,3


In [7]:
cleaned_test = cleanData(df_test)
cleaned_test

,text,label
0,im feel rather rotten im veri ambiti right,0
1,im updat blog becaus feel shitti,0
2,never make separ becaus ever want feel like asham,0
3,left bouquet red yellow tulip arm feel slightl...,1
4,wa feel littl vain thi one,0
...,...,...
1995,keep feel like someon unkind wrong think get b...,3
1996,im feel littl cranki neg thi doctor appoint,3
1997,feel use peopl give great feel achiev,1
1998,im feel comfort derbi feel though start step s...,1


In [13]:
BOW_X_train, BOW_y_train, BOW_vectorizer = createBoWFeatureVec(cleaned_data)
BOW_X_test = BOW_vectorizer.transform(cleaned_test['text']).toarray()
BOW_y_test = cleaned_test.loc[:, 'label'].values

In [15]:
TF_X2_train, TF_y_train, TF_vectorizer = createTfIdfFeatureVec(cleaned_data)
TF_X_test = TF_vectorizer.transform(cleaned_test['text']).toarray()
TF_y_test = cleaned_test.loc[:, 'label'].values

In [25]:
w = {}
for i in range(6):
    w[i]  = 1 / (np.count_nonzero(BOW_y_train == i) / len(BOW_y_train))
w

{0: 3.4290612944706385,
 1: 2.9839612085042893,
 2: 12.269938650306749,
 3: 7.410838351088468,
 4: 8.260196179659268,
 5: 27.972027972027973}

In [26]:
# USING BOW
BOW_clf, BOW_accuracy = trainLogisticRegression_weighted(BOW_X_train, BOW_X_test, BOW_y_train, BOW_y_test, w) #taking into account the class imbalance
BOW_accuracy

0.845

In [35]:
tweet_df = pd.read_csv('data/test_tweets.csv')
tweet_df = tweet_df.rename({'text': 'tweet'}, axis=1)
tweet_df = tweet_df.rename({'tokens': 'text'}, axis=1)
tweet_df = cleanData(tweet_df)
tweet_df

,Unnamed: 0,tweet,text,author,timestamp
0,4,@MarciaMarciaM12 Good thing we're from Alabama,good thing ' r alabama,ally add fb groups dms resisters onli,2021-09-30T15:57:42.000Z
1,13,"Respected @KPSingh_IAS sir, we are vocational ...",respect sir vocat trainer work 300-800 km away...,द ि ल ् ल ी व ो क े शनल ट ् र े नर,2021-09-02T07:08:08.000Z
2,25,via @NYTimes Here is an actual IDEA. Why don...,via actual idea tri come betsi devo friend,fierce mom daughter friend criminal justice re...,2020-07-19T21:35:36.000Z
3,28,GOP: “We can’t do $1200 checks. That’s irrespo...,gop check irrespons think deficit also gop par...,support queen rap also many things believe,2020-12-21T21:35:36.000Z
4,36,@JustinTrudeau Please do NOT tax my principal ...,pleas tax princip resid pleas make covid vacci...,miss canada strong free 🇨 🇦 🇨 🇦 🇨 🇦 🇨 🇦 🇨 🇦 🇨 ...,2022-01-09T00:09:16.000Z
...,...,...,...,...,...
296,1420,Got my COVID Booster yesterday and guess what?...,got covid booster yesterday guess sore arm 'm ...,bitch i'm majestic,2021-12-12T02:32:21.000Z
297,1423,My bf gave me Covid for Xmas 🙃🥴. Send snacks p...,bf gave covid xma send snack pl lol,dizzy ‍ ☼ ️ ☽ ️ ⇡ ️ chicago former emo 𝚂𝚞𝚗𝚜𝚎𝚝 ...,2021-12-19T15:07:32.000Z
298,1427,Food Grains Wishes for Our Superstar Mahesh Ba...,food grain wish superstar mahesh babu u reason...,NaN,2020-08-09T04:42:13.000Z
299,1433,@BodyForWife As long as we have Conservative g...,long conserv govern enabl transmiss therefor g...,lists blue football team political party write...,2021-07-07T15:25:12.000Z


In [37]:
tweet_text_features = BOW_vectorizer.transform(tweet_df['text']).toarray()
predictions = BOW_clf.predict(tweet_text_features)
tweet_df['predicted_emotion'] = predictions
tweet_df

,Unnamed: 0,tweet,text,author,timestamp,predicted_emotion
0,4,@MarciaMarciaM12 Good thing we're from Alabama,good thing ' r alabama,ally add fb groups dms resisters onli,2021-09-30T15:57:42.000Z,1
1,13,"Respected @KPSingh_IAS sir, we are vocational ...",respect sir vocat trainer work 300-800 km away...,द ि ल ् ल ी व ो क े शनल ट ् र े नर,2021-09-02T07:08:08.000Z,1
2,25,via @NYTimes Here is an actual IDEA. Why don...,via actual idea tri come betsi devo friend,fierce mom daughter friend criminal justice re...,2020-07-19T21:35:36.000Z,4
3,28,GOP: “We can’t do $1200 checks. That’s irrespo...,gop check irrespons think deficit also gop par...,support queen rap also many things believe,2020-12-21T21:35:36.000Z,4
4,36,@JustinTrudeau Please do NOT tax my principal ...,pleas tax princip resid pleas make covid vacci...,miss canada strong free 🇨 🇦 🇨 🇦 🇨 🇦 🇨 🇦 🇨 🇦 🇨 ...,2022-01-09T00:09:16.000Z,1
...,...,...,...,...,...,...
296,1420,Got my COVID Booster yesterday and guess what?...,got covid booster yesterday guess sore arm 'm ...,bitch i'm majestic,2021-12-12T02:32:21.000Z,0
297,1423,My bf gave me Covid for Xmas 🙃🥴. Send snacks p...,bf gave covid xma send snack pl lol,dizzy ‍ ☼ ️ ☽ ️ ⇡ ️ chicago former emo 𝚂𝚞𝚗𝚜𝚎𝚝 ...,2021-12-19T15:07:32.000Z,3
298,1427,Food Grains Wishes for Our Superstar Mahesh Ba...,food grain wish superstar mahesh babu u reason...,NaN,2020-08-09T04:42:13.000Z,1
299,1433,@BodyForWife As long as we have Conservative g...,long conserv govern enabl transmiss therefor g...,lists blue football team political party write...,2021-07-07T15:25:12.000Z,2


In [44]:
tweet_df['predicted_emotion'] = tweet_df.predicted_emotion.map(label_mapping)
tweet_df

,Unnamed: 0,tweet,text,author,timestamp,predicted_emotion
0,4,@MarciaMarciaM12 Good thing we're from Alabama,good thing ' r alabama,ally add fb groups dms resisters onli,2021-09-30T15:57:42.000Z,joy
1,13,"Respected @KPSingh_IAS sir, we are vocational ...",respect sir vocat trainer work 300-800 km away...,द ि ल ् ल ी व ो क े शनल ट ् र े नर,2021-09-02T07:08:08.000Z,joy
2,25,via @NYTimes Here is an actual IDEA. Why don...,via actual idea tri come betsi devo friend,fierce mom daughter friend criminal justice re...,2020-07-19T21:35:36.000Z,fear
3,28,GOP: “We can’t do $1200 checks. That’s irrespo...,gop check irrespons think deficit also gop par...,support queen rap also many things believe,2020-12-21T21:35:36.000Z,fear
4,36,@JustinTrudeau Please do NOT tax my principal ...,pleas tax princip resid pleas make covid vacci...,miss canada strong free 🇨 🇦 🇨 🇦 🇨 🇦 🇨 🇦 🇨 🇦 🇨 ...,2022-01-09T00:09:16.000Z,joy
...,...,...,...,...,...,...
296,1420,Got my COVID Booster yesterday and guess what?...,got covid booster yesterday guess sore arm 'm ...,bitch i'm majestic,2021-12-12T02:32:21.000Z,sadness
297,1423,My bf gave me Covid for Xmas 🙃🥴. Send snacks p...,bf gave covid xma send snack pl lol,dizzy ‍ ☼ ️ ☽ ️ ⇡ ️ chicago former emo 𝚂𝚞𝚗𝚜𝚎𝚝 ...,2021-12-19T15:07:32.000Z,anger
298,1427,Food Grains Wishes for Our Superstar Mahesh Ba...,food grain wish superstar mahesh babu u reason...,NaN,2020-08-09T04:42:13.000Z,joy
299,1433,@BodyForWife As long as we have Conservative g...,long conserv govern enabl transmiss therefor g...,lists blue football team political party write...,2021-07-07T15:25:12.000Z,love


In [45]:
tweet_df.to_csv('./data/test_tweets_pred.csv')

In [52]:
#save logisitic regression model
import pickle
with open('./models/log_reg_clf.pkl', 'wb') as f:
    pickle.dump(BOW_clf, f)
with open('./models/log_reg_vect.pkl', 'wb') as f:
    pickle.dump(BOW_vectorizer, f)